In [9]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from multiprocessing import Process
from Bio.Seq import Seq
from Bio import SeqIO
from multiprocessing import Pool

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [10]:
modelnames = ['xgb_trans_07:22:39_4_GSM4041595_K562_rep2.hg38_250_1', 'xgb_clusters_10:24:55_4_try4_250_1']
experiments = ['GSM4041591_K562_rep1_add.hg38', 'GSM4041593_K562_rep1.hg38', 'GSM4041595_K562_rep2.hg38']
cell_type = 'K562'

In [11]:
model_preds = {}

In [12]:
for modelname in modelnames:
    tables = []
    for i in experiments:
        tables.append(pd.read_csv(f'/home/pitikovegor/CourseWork/genomes/reds_experiments/K562/parsed_contacts_{i}/predict_{modelname}.tsv', sep = '\t', header = None))
    all_preds = pd.concat(tables)
    del tables
    model_preds[modelname] = all_preds

In [13]:
model_preds.keys()

dict_keys(['xgb_trans_07:22:39_4_GSM4041595_K562_rep2.hg38_250_1', 'xgb_clusters_10:24:55_4_try4_250_1'])

In [14]:
good_preds = {}

In [15]:
for modelname in modelnames:
    good_pred = model_preds[modelname][model_preds[modelname][2]>0.8]
    good_preds[modelname]= good_pred
    del good_pred

In [16]:
good_preds['xgb_clusters_10:24:55_4_try4_250_1'] = model_preds['xgb_clusters_10:24:55_4_try4_250_1'][model_preds['xgb_clusters_10:24:55_4_try4_250_1'][2]>0.99998]

In [17]:
good_preds 

{'xgb_trans_07:22:39_4_GSM4041595_K562_rep2.hg38_250_1':                                                 0         1         2
 646        D00795:28:C99BVACXX:7:1109:13547:21719  0.151441  0.848559
 20226       D00795:28:C99BVACXX:7:1115:5342:28058  0.193058  0.806942
 23645     D00795:28:C99BVACXX:7:1115:12015:100513  0.188985  0.811015
 25552       D00795:28:C99BVACXX:7:1116:2900:41883  0.149212  0.850788
 27656      D00795:28:C99BVACXX:7:1116:15591:83570  0.128948  0.871052
 ...                                           ...       ...       ...
 22067308     D00795:30:CA2UTANXX:5:1305:2910:9222  0.098569  0.901432
 22073232    D00795:30:CA2UTANXX:5:1210:8167:79111  0.148740  0.851260
 22075530   D00795:30:CA2UTANXX:5:1211:14005:80690  0.160440  0.839560
 22111781   D00795:30:CA2UTANXX:5:2310:10169:70837  0.192341  0.807659
 22115685   D00795:30:CA2UTANXX:5:2316:17538:94952  0.166956  0.833044
 
 [5509 rows x 3 columns],
 'xgb_clusters_10:24:55_4_try4_250_1':                          

In [18]:
coninfo = pd.read_csv(f'/home/pitikovegor/CourseWork/genomes/reds_experiments/{cell_type}/all_parsed/all_table.tsv', sep='\t')

In [19]:
def seq_cutter(sequence, position, chain, window = 0):
    if chain=='+':
        return str(sequence[max(0, position[0]+1-window) : min(position[1]+1+window, len(sequence))])
    else:
        return str(sequence[max(0, position[0]+1-window) : min(position[1]+1+window, len(sequence))].reverse_complement())
def open_fasta (name):
    for record in SeqIO.parse(name, "fasta"): # It's a FUCKING GENERATOR!!!!!!! I need in 1st - all of the rest is alternatively
        chr_seq = record.seq
        del record
        return chr_seq
def all_to_con(sequence, con_table):
    chr_seq = open_fasta(sequence)
    if not con_table.empty:
        con_table = con_table.apply(lambda x: [x[0], seq_cutter(chr_seq, x[2:4], x[4]), 0, 0, 0], raw = True, axis = 1, result_type='expand')
    con_table.rename(columns = {'rna_chr':'contact_seq'}, inplace = True)
    return con_table[['id', 'contact_seq']]

In [20]:
con_params = {}
for modelname in modelnames:
    good_param = coninfo[coninfo['id'].isin(good_preds[modelname].loc[:, 0])][['id', 'rna_chr', 'rna_bgn', 'rna_end', 'rna_strand']]
    con_params[modelname]= good_param
    del good_param

In [21]:
del model_preds
del good_preds

In [22]:
del coninfo

In [23]:
con_params

{'xgb_trans_07:22:39_4_GSM4041595_K562_rep2.hg38_250_1':                                               id rna_chr    rna_bgn  \
 139        D00795:30:CA2UTANXX:1:1207:7121:93342   chr19   58353438   
 309       D00795:28:C99BVACXX:8:1106:13168:79882   chr12   53307238   
 778       D00795:28:C99BVACXX:8:1313:11590:67129   chr12  125068696   
 2697      D00795:28:C99BVACXX:7:2304:12703:56768   chr12  125132328   
 15401     D00795:28:C99BVACXX:7:1215:13922:63384    chr4     441369   
 ...                                          ...     ...        ...   
 33974358   D00795:28:C99BVACXX:7:2210:6463:80418   chr18   35276003   
 33974359  D00795:30:CA2UTANXX:1:1105:12374:34586   chr18   35276058   
 33974360  D00795:28:C99BVACXX:8:2201:19808:70894   chr18   35276068   
 33976898    D00795:28:C99BVACXX:7:1116:2103:6529   chr19   56287376   
 34000487   D00795:30:CA2UTANXX:1:1205:8242:13488    chr1   52860837   
 
             rna_end rna_strand  
 139        58355683          +  
 309      

In [24]:
chr_names=['chrX', 'chrY']
for i in range(1, 23):
    chr_names.append('chr%d' % (i))
sequence_path = '/home/pitikovegor/CourseWork/genomes/chr_sequence'

In [25]:
con_params.keys()

dict_keys(['xgb_trans_07:22:39_4_GSM4041595_K562_rep2.hg38_250_1', 'xgb_clusters_10:24:55_4_try4_250_1'])

In [26]:
mod_seqs = {}
for qq in ['xgb_trans_07:22:39_4_GSM4041595_K562_rep2.hg38_250_1', 'xgb_clusters_10:24:55_4_try4_250_1']:
    #print(modelname)
    all_proc = Pool(processes = 24)
    all_data = all_proc.starmap(all_to_con, [('%s/%s.fna' % (sequence_path, i), con_params[qq][con_params[qq]['rna_chr']==i]) for i in chr_names])
    all_proc.close()
    all_proc.join()
    all_seqs = pd.concat(all_data, ignore_index = True)
    mod_seqs[qq] = all_seqs

In [27]:
mod_seqs

{'xgb_trans_07:22:39_4_GSM4041595_K562_rep2.hg38_250_1':                                                 id  \
 0            D00795:28:C99BVACXX:7:2111:9367:55490   
 1     M01460:89:000000000-AJJ2J:1:1116:22654:10850   
 2            D00795:30:CA2UTANXX:1:1202:8310:50894   
 3            D00795:30:CA2UTANXX:5:2202:3773:77078   
 4           D00795:30:CA2UTANXX:1:2211:19847:51320   
 ...                                            ...   
 4384        D00795:30:CA2UTANXX:1:1304:17878:10666   
 4385        D00795:30:CA2UTANXX:1:2112:18197:74350   
 4386  M01460:89:000000000-AJJ2J:1:2107:25915:10363   
 4387        D00795:30:CA2UTANXX:5:2215:18486:59875   
 4388        D00795:28:C99BVACXX:7:2311:10156:11899   
 
                                             contact_seq  
 0     GAGGTTATGGTTTCATGTTTGCTGTTGTTTTCTGTGGATATACATC...  
 1     TACATCTATGCCTTTGCATTGAAGAATGACTTAATTTATTCCATTC...  
 2     TAAGTTTCTAATCTCCTCATGTGTGTTTGTCAAGGGTGTGAAGTAT...  
 3     TAATTTGTAGAATTATCCAGTCATGCCAGTTGGTCCTGG

In [31]:
for i in list(mod_seqs.keys()):
    print(i)
    mod_seqs[i].to_csv(f'/home/pitikovegor/CourseWork/genomes/models/contact_seqs/{i}.tsv', sep = '\t')

xgb_trans_07:22:39_4_GSM4041595_K562_rep2.hg38_250_1
xgb_clusters_10:24:55_4_try4_250_1


In [29]:
mod_seqs.keys()

dict_keys(['xgb_trans_07:22:39_4_GSM4041595_K562_rep2.hg38_250_1', 'xgb_clusters_10:24:55_4_try4_250_1'])

# Clustering

In [1]:
import pandas as pd
import numpy as np
from multiprocessing import Process
from multiprocessing import Pool
import Levenshtein as lev
import sklearn.feature_extraction.text
from sklearn.cluster import AffinityPropagation
from Levenshtein import distance
from sklearn.feature_extraction.text import CountVectorizer
import pyopencl

In [2]:
distance('AAAAA', 'TAAAAAT')

2

In [2]:
modelname = 'xgb_trans_07:22:39_4_GSM4041595_K562_rep2.hg38_250_1'
path_to_seqs = '/home/pitikovegor/CourseWork/genomes/models/contact_seqs'

In [3]:
sequnces = pd.read_csv(f'{path_to_seqs}/{modelname}.tsv', sep = '\t')

In [7]:
sequnces = sequnces.loc[:100, :]

In [27]:
def levcounter(words_table, full_words_table):
    lev_similarity = 1/(np.array([[distance(s1,s2) for s1 in full_words_table] for s2 in words_table])+1)
    print(f'i\'m all. Table: {len(words_table)}, full teable: {len(full_words_table)}, answer shape: {lev_similarity.shape}')
    return lev_similarity

In [25]:
sequnces

,Unnamed: 0,id,contact_seq
0,0,D00795:28:C99BVACXX:7:2111:9367:55490,GAGGTTATGGTTTCATGTTTGCTGTTGTTTTCTGTGGATATACATC...
1,1,M01460:89:000000000-AJJ2J:1:1116:22654:10850,TACATCTATGCCTTTGCATTGAAGAATGACTTAATTTATTCCATTC...
2,2,D00795:30:CA2UTANXX:1:1202:8310:50894,TAAGTTTCTAATCTCCTCATGTGTGTTTGTCAAGGGTGTGAAGTAT...
3,3,D00795:30:CA2UTANXX:5:2202:3773:77078,TAATTTGTAGAATTATCCAGTCATGCCAGTTGGTCCTGGGCTTTTC...
4,4,D00795:30:CA2UTANXX:1:2211:19847:51320,TTGTCATTAGGTCTCTAGGTTCTGAGGATTACTGTGGGGTAGGCAG...
...,...,...,...
96,96,D00795:30:CA2UTANXX:1:1205:12754:68587,ATTTGAGAGTCTGTGCCTGGCTTTGTGATAGGTATAAGAAGGAGCT...
97,97,D00795:28:C99BVACXX:7:2310:3282:23813,ACAGTTTCTGTTGGATAGAACTGTGTATTCATCTGCAAACTGATAA...
98,98,D00795:28:C99BVACXX:7:1102:2640:92401,GTCCTACAAATGCCAGGCCCTGCCCATGCGGCTTCCCTCCTCAGAC...
99,99,D00795:30:CA2UTANXX:1:2315:6294:92804,CAGCCAGAAAGTCCTACAAATGCCAGGCCCTGCCCATGCGGCTTCC...


In [30]:
to_1 = int(len(sequnces['contact_seq'])/15)
all_proc_test = Pool(processes = 16)
all_data = all_proc_test.starmap(levcounter, [(sequnces['contact_seq'][i*to_1:(i+1)*to_1], sequnces['contact_seq']) for i in range(15)] + [(sequnces['contact_seq'][len(sequnces['contact_seq']) - len(sequnces['contact_seq'])%15:], sequnces['contact_seq'])])
all_proc_test.close()
all_proc_test.join()


i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 6, full teable: 101, answer shape: (6, 101)
i'm all. Table: 11, full teable: 101, answer shape: (11, 101)


In [31]:
len(sequnces['contact_seq'])%15

11

In [32]:
all_data = np.concatenate(all_data, axis=0)

In [33]:
all_data.shape

(101, 101)

In [34]:
affprop = AffinityPropagation(affinity="precomputed", damping=0.9)
affprop.fit(all_data)

/home/pitikovegor/anaconda/anaconda/lib/python3.7/site-packages/sklearn/cluster/_affinity_propagation.py:152: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 0.25 which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  FutureWarning)


AffinityPropagation(affinity='precomputed', damping=0.9)

In [ ]:
#lev_similarity = 1/(np.array([[distance(s1,s2) for s1 in sequnces['contact_seq']] for s2 in sequnces['contact_seq']])+1)
affprop = AffinityPropagation(affinity="precomputed", damping=0.9)
affprop.fit(all_data)
for cluster_id in np.unique(affprop.labels_):
    #print(cluster_id)
    #print(len(words), len(affprop.cluster_centers_indices_))
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print("Cluster center: %s\n Words in cluster: %s" % (exemplar, cluster_str))

In [29]:
vectorizer = CountVectorizer()
vectorizer

CountVectorizer()

In [30]:
X = vectorizer.fit_transform(sequnces['contact_seq'].apply(lambda x: x[500:len(x)-500]))

In [31]:
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [32]:
affprop = AffinityPropagation(damping=0.5)
affprop.fit(X)
for cluster_id in np.unique(affprop.labels_):
    print(cluster_id)
    #print(cluster_id)
    #print(len(words), len(affprop.cluster_centers_indices_))
    #exemplar = np.array(sequnces['contact_seq'])[affprop.cluster_centers_indices_[cluster_id]]
    #cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    #cluster_str = ", ".join(cluster)
   # print("Cluster center: %s\n Words in cluster: %s" % (exemplar, cluster_str))

/home/pitikovegor/anaconda/anaconda/lib/python3.7/site-packages/sklearn/cluster/_affinity_propagation.py:152: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 0.25 which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  FutureWarning)


-1


/home/pitikovegor/anaconda/anaconda/lib/python3.7/site-packages/sklearn/cluster/_affinity_propagation.py:244: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  "will not have any cluster centers.", ConvergenceWarning)


In [38]:
np.savetxt(f'{path_to_seqs}/clusters_{modelname}.tsv', affprop.labels_, delimiter='\t')

In [25]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

ModuleNotFoundError: No module named 'pycuda'